In [5]:
import pandas as pd
import numpy as np

In [6]:
FILEPATH = "/Volumes/OFFCAMPUS/Jenn/Imported data/cleaned/"
participants = ["107", "904", "905", "906"]

from ml_prep import prep_ml
X,y = prep_ml(FILEPATH, participants)

In [4]:
y.shape[0]

353

In [4]:
(n, d) = X.shape
assert n == y.shape[0]

In [5]:
# from sklearn.svm import SVC
# model = SVC(gamma='auto', kernel='linear')

In [6]:
# from sklearn.svm import LinearSVC
# model = LinearSVC(C=1e-6)

In [7]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

In [8]:
# from sklearn.linear_model import LogisticRegression
# model = LogisticRegression()

In [9]:
# from sklearn.model_selection import GridSearchCV

# param_grid = [
  # {'C': np.logspace(-10, -5, 3)}
 # ]

# tuned_model = GridSearchCV(model, param_grid)
# tuned_model.fit(X, y)

# tuned_model.cv_results_

In [10]:
from sklearn.model_selection import train_test_split

num_tries = 100
errs = np.zeros(num_tries)
for i in range(num_tries):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i+1114)
    model.fit(X_train, y_train)
    errs[i] = np.mean(model.predict(X_test) != y_test)

c:\users\karl\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [11]:
np.mean(errs)

0.4908450704225352

In [12]:
errs

array([0.49295775, 0.3943662 , 0.5915493 , 0.50704225, 0.42253521,
       0.47887324, 0.47887324, 0.57746479, 0.45070423, 0.57746479,
       0.5915493 , 0.47887324, 0.50704225, 0.45070423, 0.50704225,
       0.43661972, 0.4084507 , 0.3943662 , 0.4084507 , 0.54929577,
       0.43661972, 0.52112676, 0.50704225, 0.54929577, 0.5915493 ,
       0.50704225, 0.49295775, 0.49295775, 0.6056338 , 0.53521127,
       0.46478873, 0.6056338 , 0.53521127, 0.42253521, 0.50704225,
       0.54929577, 0.45070423, 0.30985915, 0.56338028, 0.56338028,
       0.53521127, 0.53521127, 0.53521127, 0.43661972, 0.52112676,
       0.46478873, 0.43661972, 0.6056338 , 0.52112676, 0.54929577,
       0.43661972, 0.47887324, 0.50704225, 0.47887324, 0.54929577,
       0.50704225, 0.45070423, 0.52112676, 0.47887324, 0.49295775,
       0.36619718, 0.49295775, 0.46478873, 0.4084507 , 0.45070423,
       0.42253521, 0.43661972, 0.3943662 , 0.42253521, 0.46478873,
       0.6056338 , 0.42253521, 0.38028169, 0.49295775, 0.56338